# S03 - Structures de données Python I - Exemples logistique

Voici quelques exemples de chaîne logistique :
1. Méthodes de prévision
- Naïve (promenade aléatoire et saisonnière)
- Moyenne mobile
- Lissage exponentiel simple
2. Mesures d'erreur de prévision : erreur quadratique moyenne (MSE), erreur absolue moyenne en pourcentage (MAPE)
3. Recherche de paramètres optimaux pour les méthodes de prévision à l'aide de Python

*REMARQUE :* les exemples sont utilisés pour montrer des applications de structures de données abordées dans la séance 3. Il est également possible d'utiliser une structure de données plus avancée (en particulier un DataFrame) pour cela (qui sera abordée à la séance 4).

## Exemple 1 : Méthode de prévision naïve
Dans la méthode de prévision naïve simple, nous utilisons simplement la demande observée de la période précédente $t$ comme prévision pour la période $t+1$. Formellement, nous avons

> $$F_{t+1}=D_{t}$$
> Où :
> - $F_{t+1}$ : prévision pour la période $t+1$
> - $D_t$ : demande pour la période $t$

Cette méthode est appelée prévisions de « marche aléatoire » et fonctionne étonnamment bien pour prédire certaines séries temporelles financières (par exemple, les cours des actions).

Compte tenu des ventes historiques ci-dessous des 12 derniers mois
- Ventes historiques = `[125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]`

Nous pouvons écrire une fonction pour calculer la prévision naïve d'une période donnée $t$.

In [1]:
# nous préparons d'abord la liste des demandes historiques (ventes)
sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]

In [2]:
# définition de la fonction de prévision naïve
def naiveForecast(historical_sales, t):
    """
    Return the predicted demand for the next period using the naive forecast
    parameters:
        historical_sales: (list) real sales in the previous periods
        t: (int number) period to forecast
    return:
        forecast for period t
    """
    
    return historical_sales[t-1]

In [3]:
t = 12 # il s'agit de la période 13 car le premier index commence à zéro. Nous ne pouvons pas aller au-delà car nous n'avons que 12 éléments dans la liste (indexés 0 -> 11)
print("Naive forecast for t = ", t, " is ", "{:.2f}".format(naiveForecast(sales, t)))  # l'utilisation de "{:.2f}".format(x) permet d'imprimer exactement 2 chiffres

Naive forecast for t =  12  is  126.00


Alternativement, dans la prévision naïve **saisonnière**, nous pouvons revenir à la même période du cycle de saisonnalité précédent et prendre cette valeur comme prévision (par exemple, en utilisant les ventes de janvier 2021 comme prévision pour janvier 2022). Nous pouvons étendre la fonction précédente pour ce cas.

In [4]:
# définition de la fonction de prévision naïve
def seasonalNaiveForecast(historical_sales, t, s):
    """
    Return the predicted demand for the next period using the seasonal naive forecast
    parameters:
        historical_sales: (list) real sales in the previous periods
        t: (int number) period to forecast
        s: (int number) length of seasonality cycle
    return:
        forecast for period t
    """
    
    return historical_sales[t-s]

In [5]:
t = 12 # période de prévision
s = 6 # cycle de saisonnalité
print("Seasonal Naive forecast for t = ", t, "and s =", s," is ", "{:.2f}".format(seasonalNaiveForecast(sales, t,s)))

Seasonal Naive forecast for t =  12 and s = 6  is  128.00


## Exemple 2 : Méthode de prévision par moyenne mobile

La moyenne mobile est une méthode de séries chronologiques qui utilise les données sur la demande passée pour prédire la demande future. Comme d'autres méthodes de séries chronologiques, la moyenne mobile repose sur l'hypothèse que l'histoire se répète et, par conséquent, elle ignore les changements de l'environnement qui peuvent affecter la demande future.

**Brève description du modèle :** cette méthode consiste à calculer la demande moyenne pour les $k$ périodes les plus récentes et à l'utiliser comme prévision de la demande. Cette méthode peut être représentée à l'aide de l'équation suivante :
$$F_{t+1}=\frac{D_t+D_{t-1}+D_{t-2}+...+D_{(t-k)+1}}{k}$$
Où:
- $k$ : nombre d'observations utilisées dans le calcul

Par exemple, si k = 3, la prévision de la période $21$ peut être calculée par : $F_{21}=\frac{D_{20}+D_{19}+D_{18}}{3}$.
Le choix du nombre de périodes ($k$) à considérer pour réaliser la prévision de la demande est important :
- Si $k$ est petit, la prévision réagira rapidement aux changements réels (c'est-à-dire aux variations de la demande qui ne sont pas aléatoires), mais elle sera également davantage influencée par les variations aléatoires ;
- si $k$ est grand, les prévisions seront moins affectées par les variations aléatoires de la demande, mais aussi plus lentes à réagir aux changements réels.

Étant donné une liste avec des ventes historiques et la valeur de $k$, nous voulons prédire le volume des ventes pour la prochaine période.
- Prédire les ventes futures en utilisant $k=2$
- Prédire les ventes futures en utilisant $k=3$

In [6]:
# définition de la fonction moyenne mobile
def movingAvg(historical_sales, t, k):
    """
    Return the predicted demand for the next period
    parameters:
        historical_sales: (list) real sales in the previous periods
        t: (int number) period to forecast
        k: (int number) parameter of the moving avg method
    return:
        forecast for period t
    """
    return sum(historical_sales[t-k:t]) / k

t = 12 # période de prévision

print("Moving average forecast (k = 2) for t = ", t, " is ", "{:.2f}".format(movingAvg(sales, t, 2)))
print("Moving average forecast (k = 3) for t = ", t, " is ", "{:.2f}".format(movingAvg(sales, t, 3)))

Moving average forecast (k = 2) for t =  12  is  129.00
Moving average forecast (k = 3) for t =  12  is  130.67


## Exemple 3 : Méthode de prévision par lissage exponentiel

En utilisant une prévision de la demande initiale, un facteur de lissage $0 \leq \alpha \leq 1$, nous pouvons calculer la prévision en utilisant le paramètre de lissage comme

$$ F_{t+1}=\alpha D_t + (1-\alpha) F_t $$

Où:
- $\alpha$ = choix du paramètre de lissage.

Par conséquent, nous devons d’abord calculer $F_t$ qui est fondamentalement égal à
$ F_{t}=\alpha D_{t-1} + (1-\alpha) F_{t-1}$.


Ainsi, nous devrons calculer la première prévision (période de prévision 2 à partir de la période 1) et calculer *itérativement* la prévision de la période 2 jusqu'à $t + 1$.

Étant donné une liste avec des ventes historiques et la valeur de $\alpha$, nous pouvons créer une fonction qui utilise la méthode de lissage exponentiel simple pour calculer la demande prévue pour une période donnée.
- Prédire les ventes futures en utilisant $\alpha= 0.2$
- Prédire les ventes futures en utilisant $\alpha= 0.5$

In [7]:
# définition de la fonction de lissage exponentiel
def exponentialSmoothing(historical_sales, t, alpha):
    """
    Return the predicted demand for the next period
    parameters:
        historical_sales: (list) real sales in the previous periods
        t: (int number) period to forecast
        alpha: smoothing parameter
    return:
        forecast for period t
    """
    exp_forecast = [] # commencer avec une liste vide

    # Nous pouvons utiliser list.append() pour ajouter un élément à la liste
    exp_forecast.append(historical_sales[0]) # supposons que la prévision initiale (indice 0) = demande réelle sur la même période
    # calculer les prévisions pour la période 1 à t de manière itérative
    for i in range(1, t+1):
        f = alpha * historical_sales[i-1] + (1 - alpha) * exp_forecast[i-1]
        exp_forecast.append(f)

    # print("exp_forecast list: ", exp_forecast) # vous pouvez commenter ceci pour afficher la liste produite
    return  exp_forecast[t]

t = 12 # période de prévision
print("Exponential smoothing forecast (alpha = 0.2) for t = ", t, " is ", "{:.2f}".format(exponentialSmoothing(sales, t, 0.2)))
print("Exponential smoothing forecast (alpha = 0.5) for t = ", t, " is ", "{:.2f}".format(exponentialSmoothing(sales, t, 0.5)))

Exponential smoothing forecast (alpha = 0.2) for t =  12  is  131.54
Exponential smoothing forecast (alpha = 0.5) for t =  12  is  129.37


## Exemple 4 : Mesures de prévision

**Erreur quadratique moyenne (MSE)**

Différentes méthodes de prévision peuvent fournir une qualité de prévision différente. Afin d'estimer la qualité d'une prévision, certaines mesures sont utilisées dans la pratique, notamment l'erreur quadratique moyenne (MSE).

Le MSE mesure l'écart quadratique entre les données prévues et réelles selon l'équation suivante.

$$ MSE = \frac{1}{T}\sum_{t=1}^{T}(D_t-F_t)^2$$

Étant donné deux entrées : (i) une liste de prévisions de la demande et (ii) une liste de réalisations de la demande, créez une fonction qui renvoie le MSE. Tout d'abord, nous définissons la fonction.

In [8]:
# définir une fonction qui calcule le MSE
def MSE(forecast, real_demand):
    """
    Compute the mean squared error 
    parameters:
        forecast: (list) demand forecast for a given planning horizon
        real_demand: (list) real demand over a given planning horizon
        Attention: the lists of real_demand and forecast must be of the same size
    return:
        MSE: (number) mean squared error
    """
    sum_mse = 0
    n_periods = len(forecast)  # obtenir le nombre de périodes de la liste
    for t in range(n_periods):
        sum_mse += (real_demand[t] - forecast[t]) ** 2
    return sum_mse / n_periods

Ensuite, nous appelons la fonction pour connaître le MSE de notre prévision.

In [9]:
# Considérez une liste de ventes réelles et de prévisions
real_sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]
predictions = [121, 132, 110, 133, 146, 132, 128, 115, 136, 132, 130, 125]

print("The MSE of our predictions is: ", "{:.2f}".format(MSE(predictions, real_sales)))

The MSE of our predictions is:  62.83


Alternativement, nous pouvons également utiliser la *compréhension de liste* au lieu de la boucle *for*

In [10]:
# définir une fonction qui calcule le MSE en utilisant la compréhension de liste
def MSE(forecast, real_demand):
    n_periods = len(forecast)
    mse_t = [(real_demand[t] - forecast[t]) ** 2 
             for t in range(n_periods)] # MSE pour chaque période de temps t
    return sum(mse_t) / n_periods # on utilise la fonction sum() pour calculer la somme de mse_t

# Considérez une liste de ventes réelles et de prévisions
real_sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]
predictions = [121, 132, 110, 133, 146, 132, 128, 115, 136, 132, 130, 125]

print("The MSE of our predictions is: ", "{:.2f}".format(MSE(predictions, real_sales)))

The MSE of our predictions is:  62.83


**Erreur absolue moyenne en pourcentage (MAPE)**

Une autre mesure de prévision couramment utilisée est le MAPE qui peut être calculé comme suit

$$ MAPE = \frac{1}{T}\sum_{t=1}^{T}\left|\frac{D_t-F_t}{D_t}\right|$$

et peut être mis en œuvre comme

In [11]:
# définir une fonction qui calcule MAPE en utilisant la compréhension de liste
def MAPE(forecast, real_demand):
    """
    Compute the mean absolute percentage error 
    parameters:
        forecast: (list) demand forecast for a given planning horizon
        real_demand: (list) real demand over a given planning horizon
        Attention: the lists of real_demand and forecast must be of the same size
    return:
        Mean absolute percentage errors (MAPE)
    """
    n_periods = len(forecast)
    mape_t = [abs(real_demand[t] - forecast[t]) / real_demand[t] 
              for t in range(n_periods)] 
    return sum(mape_t) / n_periods # on utilise la fonction sum() pour calculer la somme de mape_t

# Considérez une liste de ventes réelles et de prévisions
real_sales = [125, 142, 120, 153, 156, 135, 128, 117, 140, 134, 132, 126]
predictions = [121, 132, 110, 133, 146, 132, 128, 115, 136, 132, 130, 125]

print("The MAPE of our predictions is: ", "{:.2f}".format(MAPE(predictions, real_sales) * 100),"%")

The MAPE of our predictions is:  4.05 %


## Exemple 5 : Prévision et mesure des performances

Nous utilisons ici toutes les méthodes ensemble. Nous allons essayer différents modèles de prévision avec différentes configurations et mesurer leurs performances. Les résultats seront stockés dans un dictionnaire (imbriqué).

Nous utiliserons les méthodes définies précédemment et trouverons la meilleure configuration pour chaque méthode lors de la prévision des prix du United States Oil Fund (USO) qui suit le prix du West Texas Intermediate Light Sweet Crude en 2019. Les prévisions et les erreurs seront calculées pour le second semestre 2019 (c'est-à-dire les éléments avec l'indice 6-11) pour être mesurées par rapport aux valeurs réelles.

In [12]:
uso_2019 = [90.80, 95.60, 100.00, 106.32, 88.80, 96.32, 96.31, 91.68, 90.72, 90.40, 92.96, 102.48]

Premièrement, la méthode naïve comme modèle de base

In [13]:
uso_fcst_naive = [naiveForecast(uso_2019, t) for t in range(6, 12)]
print(uso_fcst_naive)

uso_fcst_naive_mse = MSE(uso_fcst_naive, uso_2019[6:])
print("MSE = ", uso_fcst_naive_mse)

uso_fcst_naive_mape = MAPE(uso_fcst_naive, uso_2019[6:])
print("MAPE = ", uso_fcst_naive_mape)

# Nous pouvons ensuite conserver les résultats dans un dictionnaire
fcst_results = {}  # créez d'abord un dictionnaire vide
fcst_names = ['Naive', 'SeasonNaive', 'MovingAvg', 'ExpSmooth'] # liste des méthodes de prévision
print("add results to the dictionary for the method: ", fcst_names[0])
fcst_results[fcst_names[0]] = {'Forecast': uso_fcst_naive, 
                               'MSE': uso_fcst_naive_mse, 
                               'MAPE': uso_fcst_naive_mape}
print(fcst_results)

[96.32, 96.31, 91.68, 90.72, 90.4, 92.96]
MSE =  19.940833333333348
MAPE =  0.030860385227782578
add results to the dictionary for the method:  Naive
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}}


Ensuite, nous appliquons la prévision naïve saisonnière et recherchons la meilleure durée de saisonnalité $s$ entre 2, 4 et 6 basée sur le MAPE.

In [14]:
best_s = -1  # initialiser la valeur initiale du meilleur s
best_mape = 1.0  # initialiser la valeur initiale de MAPE (au maximum 100%)

for s in range(2,7,2):  # ici, nous pouvons utiliser range(start, end, step) pour créer un incrément de 2, ou, alternativement, utiliser la liste [2,4,6]
    print("s = ", s)
    uso_fcst_seasonnaive = [seasonalNaiveForecast(uso_2019, t, s) for t in range(6, 12)]
    uso_fcst_seasonnaive_mape = MAPE(uso_fcst_seasonnaive, uso_2019[6:])
    print("MAPE :", uso_fcst_seasonnaive_mape)
    if uso_fcst_seasonnaive_mape < best_mape:  # garder une trace des meilleurs s
        best_s = s  # définir les nouveaux best_s
        best_mape = uso_fcst_seasonnaive_mape  # définir le nouveau best_mape

# nous calculons à nouveau les prévisions et les résultats correspondants en fonction du meilleur paramètre
print("best seasonality length s = ", best_s)
uso_fcst_seasonnaive = [seasonalNaiveForecast(uso_2019, t, best_s) for t in range(6, 12)]
uso_fcst_seasonnaive_mse = MSE(uso_fcst_seasonnaive, uso_2019[6:])
uso_fcst_seasonnaive_mape = MAPE(uso_fcst_seasonnaive, uso_2019[6:])

print("add results to the dictionary for the method: ", fcst_names[1])
fcst_results[fcst_names[1]] = {'Forecast': uso_fcst_seasonnaive, 
                               'MSE': uso_fcst_seasonnaive_mse, 
                               'MAPE': uso_fcst_seasonnaive_mape}
print(fcst_results)

s =  2
MAPE : 0.057723114537770515
s =  4
MAPE : 0.07101230185854347
s =  6
MAPE : 0.08053786497565112
best seasonality length s =  2
add results to the dictionary for the method:  SeasonNaive
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}, 'SeasonNaive': {'Forecast': [88.8, 96.32, 96.31, 91.68, 90.72, 90.4], 'MSE': 43.62669999999999, 'MAPE': 0.057723114537770515}}


Nous appliquons maintenant la prévision de la moyenne mobile et recherchons la meilleure période de rétrospection $k$ entre 2 et 5 en fonction de MAPE.

In [15]:
best_k = 0  # initialiser la valeur initiale du meilleur k
best_mape = 1.0  # initialiser la valeur initiale de MAPE (au maximum 100%)

for k in range(2,6):
    print("k = ", k)
    uso_fcst_movingavg = [movingAvg(uso_2019, t, k) for t in range(6, 12)]
    uso_fcst_movingavg_mape = MAPE(uso_fcst_movingavg, uso_2019[6:])
    print("MAPE :", uso_fcst_movingavg_mape)
    if uso_fcst_movingavg_mape < best_mape:  # garder une trace du meilleur paramètre
        best_k = k  # définir le nouveau best_k
        best_mape = uso_fcst_movingavg_mape  # définir le nouveau best_mape

# nous calculons à nouveau les prévisions et les résultats correspondants en fonction du meilleur paramètre
print("best lookback length k = ", best_k)
uso_fcst_movingavg = [movingAvg(uso_2019, t, best_k) for t in range(6, 12)]
uso_fcst_movingavg_mse = MSE(uso_fcst_movingavg, uso_2019[6:])
uso_fcst_movingavg_mape = MAPE(uso_fcst_movingavg, uso_2019[6:])

print("add results to the dictionary for the method: ", fcst_names[2])
fcst_results[fcst_names[2]] = {'Forecast': uso_fcst_movingavg, 
                               'MSE': uso_fcst_movingavg_mse, 
                               'MAPE': uso_fcst_movingavg_mape}
print(fcst_results)

k =  2
MAPE : 0.044274444653136145
k =  3
MAPE : 0.03909421269515928
k =  4
MAPE : 0.042306982792044284
k =  5
MAPE : 0.04302070201943891
best lookback length k =  3
add results to the dictionary for the method:  MovingAvg
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}, 'SeasonNaive': {'Forecast': [88.8, 96.32, 96.31, 91.68, 90.72, 90.4], 'MSE': 43.62669999999999, 'MAPE': 0.057723114537770515}, 'MovingAvg': {'Forecast': [97.14666666666666, 93.81, 94.77, 92.90333333333335, 90.93333333333334, 91.36], 'MSE': 25.944644444444453, 'MAPE': 0.03909421269515928}}


Enfin, nous appliquons la prévision de lissage exponentiel et recherchons le meilleur $\alpha$ entre 0,1 et 0,9 avec un pas de 0,1 basé sur MAPE.

In [16]:
best_alpha = 0  # initialiser la valeur initiale du meilleur alpha
best_mape = 1.0  # initialiser la valeur initiale de MAPE (au maximum 100%)

for alpha in [0.1 * i for i in range(1,10)]:  # créer une liste de [0,1,...,0,8] en utilisant la compréhension de liste
    print("alpha = ", "{:.1f}".format(alpha))
    uso_fcst_expsmooth = [exponentialSmoothing(uso_2019, t, alpha) for t in range(6, 12)]
    uso_fcst_expsmooth_mape = MAPE(uso_fcst_expsmooth, uso_2019[6:])
    print("MAPE :", uso_fcst_expsmooth_mape)
    if uso_fcst_expsmooth_mape < best_mape:  # garder une trace du meilleur paramètre
        best_alpha = alpha  # définir le nouveau meilleur paramètre
        best_mape = uso_fcst_expsmooth_mape  # définir le nouveau best_mape

# nous calculons à nouveau les prévisions et les résultats correspondants en fonction du meilleur paramètre
print("best alpha = ", best_alpha)
uso_fcst_expsmooth = [exponentialSmoothing(uso_2019, t, best_alpha) for t in range(6, 12)]
uso_fcst_expsmooth_mse = MSE(uso_fcst_expsmooth, uso_2019[6:])
uso_fcst_expsmooth_mape = MAPE(uso_fcst_expsmooth, uso_2019[6:])

print("add results to the dictionary for the method: ", fcst_names[3])
fcst_results[fcst_names[3]] = {'Forecast': uso_fcst_expsmooth, 
                               'MSE': uso_fcst_expsmooth_mse, 
                               'MAPE': uso_fcst_expsmooth_mape}
print(fcst_results)

alpha =  0.1
MAPE : 0.03455040544430794
alpha =  0.2
MAPE : 0.03740061515515078
alpha =  0.3
MAPE : 0.038183022048744
alpha =  0.4
MAPE : 0.038291632918183605
alpha =  0.5
MAPE : 0.03781851315454915
alpha =  0.6
MAPE : 0.03707021866224728
alpha =  0.7
MAPE : 0.03611894256920158
alpha =  0.8
MAPE : 0.034869814598145234
alpha =  0.9
MAPE : 0.03315885402784039
best alpha =  0.9
add results to the dictionary for the method:  ExpSmooth
{'Naive': {'Forecast': [96.32, 96.31, 91.68, 90.72, 90.4, 92.96], 'MSE': 19.940833333333348, 'MAPE': 0.030860385227782578}, 'SeasonNaive': {'Forecast': [88.8, 96.32, 96.31, 91.68, 90.72, 90.4], 'MSE': 43.62669999999999, 'MAPE': 0.057723114537770515}, 'MovingAvg': {'Forecast': [97.14666666666666, 93.81, 94.77, 92.90333333333335, 90.93333333333334, 91.36], 'MSE': 25.944644444444453, 'MAPE': 0.03909421269515928}, 'ExpSmooth': {'Forecast': [95.736392, 96.2526392, 92.13726392000001, 90.861726392, 90.44617263920001, 92.70861726392], 'MSE': 20.876522040985435, 'MAPE

Nous pouvons ensuite imprimer tous les résultats sur plusieurs lignes pour MSE et MAPE. La méthode naïve s'avère être la meilleure pour cette série temporelle, suivie du lissage exponentiel avec $\alpha = 0,9$ :

In [17]:
for method in fcst_names:
    print("Method:", method, 
          ", MSE = ", "{:.2f}".format(fcst_results[method]['MSE']),
          ", MAPE = ", "{:.2f}".format(fcst_results[method]['MAPE']*100),"%")

Method: Naive , MSE =  19.94 , MAPE =  3.09 %
Method: SeasonNaive , MSE =  43.63 , MAPE =  5.77 %
Method: MovingAvg , MSE =  25.94 , MAPE =  3.91 %
Method: ExpSmooth , MSE =  20.88 , MAPE =  3.32 %


**Supplément :** nous pouvons également utiliser un processus similaire pour « combiner » deux (ou même plusieurs) prévisions en utilisant le poids $0 \leq w \leq 1$ qui peut être optimisé. Par exemple, nous pouvons combiner les meilleurs SeasonNaive et MovingAvg, c'est-à-dire $w\times SeasonNaive + (1-w)\times MovingAvg$ comme suit :

In [18]:
best_w = -1  # initialiser la valeur initiale du meilleur w
best_mape = 1.0  # initialiser la valeur initiale de MAPE (au maximum 100%)

for w in [0.1 * i for i in range(10)]:  # créer une liste de [0.0,...,1.0] en utilisant la compréhension de liste
    print("aggregation weight w = ", "{:.1f}".format(w))
    uso_fcst_combined = [w * fcst_results['SeasonNaive']['Forecast'][t] + 
                         (1 - w) * fcst_results['MovingAvg']['Forecast'][t] 
                         for t in range(6)]
    # imprimer(uso_fcst_combined)
    uso_fcst_combined_mape = MAPE(uso_fcst_combined, uso_2019[6:])

    print("MAPE :", uso_fcst_combined_mape)
    if uso_fcst_combined_mape < best_mape:  # garder une trace du meilleur paramètre
        best_w = w  # définir le nouveau meilleur paramètre
        best_mape = uso_fcst_combined_mape  # définir le nouveau best_mape

aggregation weight w =  0.0
MAPE : 0.03909421269515928
aggregation weight w =  0.1
MAPE : 0.03835785738743272
aggregation weight w =  0.2
MAPE : 0.04050339965582033
aggregation weight w =  0.3
MAPE : 0.04265586401606416
aggregation weight w =  0.4
MAPE : 0.04480832837630788
aggregation weight w =  0.5
MAPE : 0.04696079273655168
aggregation weight w =  0.6
MAPE : 0.049113257096795426
aggregation weight w =  0.7
MAPE : 0.05126572145703925
aggregation weight w =  0.8
MAPE : 0.053418185817282936
aggregation weight w =  0.9
MAPE : 0.05557065017752679


Vous pouvez voir que le meilleur MAPE (3,8 %) avec $w = 0,1$ est en fait meilleur que le meilleur des deux méthodes (voir $w=0,0$ et $w=1,0$) dans ce cas.